## Preparing the Execution Environment

For this tutorial we are required to update the existing default Parsl package with Parsl that has the integration files (RP-Parsl integration will be relased in Parsl soon).

In [ ]:
%%capture capt

# remove the exisitng parsl from conda
!conda remove --force parsl -y

# install a specific parsl version
!pip install git+https://github.com/AymenFJA/parsl.git@master

Next we need to locate the installed `nwchem` executable in our environment

In [ ]:
# locate the nwchem executable path
nwchem_path = !which nwchem
nwchem      = nwchem_path[0]

Gather the MongoDB server information and set the `RADICAL_PILOT_DBURL` environment variable.

In [2]:
%%capture capt

import os

mdb_host = os.environ.get('MDB_SERVER', 'mongodb')
mdb_port = os.environ.get('MDB_PORT',   '27017')
mdb_name = os.environ.get('MDB_NAME',   'guest')
mdb_pswd = os.environ.get('MDB_PSWD',   'guest')
mdb_dtbs = os.environ.get('MDB_DTBS',   'default')

%env RADICAL_PILOT_DBURL=mongodb://$mdb_name:$mdb_pswd@$mdb_host:$mdb_port/$mdb_dtbs

# MPI Nwchem Example:

The following example application shows the execution of MP2 geometry optimization followed by a CCSD(T) energy evaluation at the converged geometry. A Dunning correlation-consistent triple-zeta basis is used. The default of Cartesian basis functions must be overridden using the keyword spherical on the BASIS directive. The 1s core orbitals are frozen in both the MP2 and coupled-cluster calculations (note that these must separately specified).

First, we need to write the `nwchem` example to a file so we can use it provide it as an input to the `nwchem` executable later.

In [ ]:
input = """
start n2

geometry
  symmetry d2h
  n 0 0 0.542
end

basis spherical
  n library cc-pvtz
end

mp2
  freeze core
end

task mp2 optimize

ccsd
  freeze core
end

task ccsd(t)
"""
nwchem_input = '{0}/{1}'.format(os.getcwd(), 'mp2_optimization.nw')
with open(nwchem_input,"w+") as f:
    f.writelines(input)

Now, we import parsl Python module in our application and the RadicalPilotExecutor

In [3]:
import os
import parsl
import radical.pilot as rp

from parsl.config import Config
from parsl.app.app import python_app, bash_app
from parsl.executors import RadicalPilotExecutor

Set the path of RadicalPilotExecutor configuration file. The rpex.cfg is a configuraion file that describes the resource
distribuation between functions and executables for RadicalPilotExecutor and pass it to Parsl dataflow configuration with RadicalPilotExecutor to load it. 

In [ ]:
rpex_cfg = 'rpex.cfg'
config = Config(
    executors=[RadicalPilotExecutor(
        rpex_cfg=rpex_cfg, bulk_mode=True,
        resource='local.localhost', login_method = 'local',
        walltime=30, managed= True, cores= 4
    )])

parsl.load(config)

Create a simple Parsl `@bashapp` to invoke the `nwchem` task. The `bashapp` requires the type of the task which in this case it is an `MPI` and the number of cpu_processes (4 MPI ranks where each rank takes 1 core) to run on.

In [ ]:
@bash_app
def nwchem_mp2_optimization(cpu_processes=2, cpu_process_type=rp.MPI):

    return "{0} {1}".format(nwchem, nwchem_input)

# invoke the nwchem_mp2_optimization
future  = nwchem_mp2_optimization()

# wait for the results of the nwchem task.
if future.result() == 0:
        print('Parsl task {0} finished'.format(future.tid))    